In [ ]:
import xarray as xr
import os

import rioxarray
import matplotlib.pyplot as plt
import rasterio
import datetime

This is a process to stack up rasters to the extent of each training VIIRS image, matching the resolution of the highest resolution product 

In [ ]:
dem_path = '../Data/training/LANDFIRE/LC20_Elev_220.tif'
dem = rioxarray.open_rasterio(dem_path)
#xr.open_dataset(dem_path)

In [ ]:
bands = ['CMI_C0' + str(i) for i in range(1,10)]
bands.extend(['CMI_C' + str(i) for i in range(10,17)])
bands.append('goes_imager_projection')


goes_path = '../Data/training/GOES/2017/GOES_2017-06-15-08.nc'
# goes = xr.open_dataset(goes_path)[bands]

In [ ]:
viirs_path = '../Data/training/VIIRS/Rasterized/COG/2017/bool/VIIRS_Rasterized_2017-03-07_7_bool.tif'
old_viirs = xr.open_dataset(viirs_path)

# Code to clip GOES imagery to VIIRS data

Actually have to read goes in with rioxarray.open_rasterio()

In [ ]:
viirs_path = '../Data/Training/VIIRS/Rasterized/Individual_Fires/variable_size/2017/bool/VIIRS_Rasterized_2017-06-15_8_Num3_bool.nc'
viirs = xr.open_dataset(viirs_path)
viirs = viirs.rio.write_crs('EPSG:4326')


In [ ]:
goes_rio = rioxarray.open_rasterio(goes_path)
match = goes_rio.rio.reproject_match(viirs.fillna(0))


In [ ]:
match_viirs = match.assign({'fire_bool':viirs.bool})

# vars_list = list(match_viirs.data_vars)  
# bads = []
# for var in vars_list:  
# #     print(goes_viirs[var].attrs)
#     try:
#         del match_viirs[var].attrs['grid_mapping']
#     except KeyError:
#         bads.append(var)
        
# match_viirs
# match_viirs.to_netcdf("temp/matched_plus_viirs.nc")

In [ ]:
viirs.rio.reproject(dem.rio.crs).rio.bounds()

In [ ]:
bounds = (-1172029.42686689, 1208355.3423637336, -1135661.900541234, 1245221.0539815216)

dem_viirs = dem.rio.clip_box(bounds[0],bounds[1],bounds[2],bounds[3]).rio.reproject_match(viirs.rio.reproject("EPSG:4326"))

In [ ]:
dem_viirs

In [ ]:
dem_goes = goes_rio.rio.reproject_match(dem_viirs)

In [ ]:
three_matched = dem_goes.assign({'dem': dem_viirs,'fire_bool':viirs.bool})


In [ ]:
vars_list = list(three_matched.data_vars)  
bads = []
for var in vars_list:  
#     print(goes_viirs[var].attrs)
    try:
        del three_matched[var].attrs['grid_mapping']
    except KeyError:
        bads.append(var)
        
three_matched.to_netcdf("temp/three_matched.nc")

# Pseudo code for complete stacking / regridding pipeline

Write:
- Function that takes VIIRS file, takes the metadata from file name, and outputs the corresponding GOES file
- Small function that returns the DEM-readible bounding box of a VIIRS image
- Function that will perform feature engineering on the GOES imagery

In [ ]:
# Check if 2017 lines up as expected
def get_files(parent_dir, suffix):
    files = []
    for r,d,f in os.walk(parent_dir):
        for file in f:
            if file.endswith(suffix):
                files.append(os.path.join(r,file))

                
    return files


In [ ]:
def get_goes_from_viirs(viirs_file):
    _split = viirs_file.split('_')
    date, hour, num = _split[4:7]
    samp_goes = list('/Users/seancarter/Documents/Data/Training/GOES/clipped/2017/GOES_2017-03-01-17_Num2.nc')
    
    year = date[:4]
    
    samp_goes[-21:] = f'{date}-{hour}_{num}.nc'
    samp_goes[-31:-27] = year
    return ('').join(samp_goes)

In [ ]:
def get_goes_time(goes_path):

    split_1 = goes_path.split('/')[-1].split('-')

    year = int(split_1[0].split('_')[-1])

    month, day = int(split_1[1]), int(split_1[2])

    hour = int(split_1[-1].split('.')[0].split('_')[0])

    date = datetime.datetime(year, month, day, hour)

    return date

In [ ]:
goes_files[0].split('/')[-1].split('-')[-1].split('.')[0]

In [ ]:
get_goes_time(goes_files[:5])

In [ ]:
goes_files = get_files('/Users/seancarter/Documents/Data/Training/GOES/clipped/','.nc')
# goes_files.extend(get_files('/Users/seancarter/Documents/Data/Training/GOES/clipped/2018/','.nc'))
# goes_files.extend(get_files('/Users/seancarter/Documents/Data/Training/GOES/clipped/2019/','.nc'))

viirs_files = get_files('/Users/seancarter/Documents/Data/Training/VIIRS/Rasterized/Individual_Fires/variable_size/2017/bool/','.nc')
viirs_files.extend(get_files('/Users/seancarter/Documents/Data/Training/VIIRS/Rasterized/Individual_Fires/variable_size/2018/bool/','.nc'))
viirs_files.extend(get_files('/Users/seancarter/Documents/Data/Training/VIIRS/Rasterized/Individual_Fires/variable_size/2019/bool/','.nc'))
viirs_files.extend(get_files('/Users/seancarter/Documents/Data/Training/VIIRS/Rasterized/Individual_Fires/variable_size/2020/bool/','.nc'))
viirs_files.extend(get_files('/Users/seancarter/Documents/Data/Training/VIIRS/Rasterized/Individual_Fires/variable_size/2021/bool/','.nc'))



In [ ]:
len(goes_files)

In [ ]:
len(viirs_files)

In [ ]:
goes_files.sort()
viirs_files.sort()

In [ ]:
# Load in all landfire data
dem = '/Users/seancarter/Documents/Data/Training/LANDFIRE/LF2020_Elev_220_CONUS/Tif/LC20_Elev_220.tif'
cbd = '/Users/seancarter/Documents/Data/Training/LANDFIRE/LF2022_CBD_220_CONUS/Tif/LC22_CBD_220.tif'
evc = '/Users/seancarter/Documents/Data/Training/LANDFIRE/LF2022_EVC_220_CONUS/Tif/LC22_EVC_220.tif'
evh = '/Users/seancarter/Documents/Data/Training/LANDFIRE/LF2022_EVH_220_CONUS/Tif/LC22_EVH_220.tif'
f40 = '/Users/seancarter/Documents/Data/Training/LANDFIRE/LF2022_FBFM40_220_CONUS/Tif/LC22_F40_220.tif'
fvh = '/Users/seancarter/Documents/Data/Training/LANDFIRE/LF2022_FVH_220_CONUS/Tif/LC22_FVH_220.tif'
landfire_list = [dem, cbd, evc, evh, f40, fvh]
dem, cbd, evc, evh, f40, fvh = [rioxarray.open_rasterio(file) for file in landfire_list]

In [ ]:
viirs_bool_fires = get_files('/Users/seancarter/Documents/Data/Training/VIIRS/Rasterized/Individual_Fires/variable_size/2020/bool/','.nc')
viirs_frp_fires = get_files('/Users/seancarter/Documents/Data/Training/VIIRS/Rasterized/Individual_Fires/variable_size/2020/frp/','.nc')

In [ ]:
bool_img = xr.open_dataset(viirs_bool_fires[0])
bool_img = bool_img.rio.write_crs('EPSG:4326')

frp_img = xr.open_dataset(viirs_frp_fires[0])
frp_img = frp_img.rio.write_crs('EPSG:4326')


goes = rioxarray.open_rasterio(get_goes_from_viirs(viirs_bool_fires[0]))

bounds = bool_img.rio.reproject(dem.rio.crs).rio.bounds()

landfire1 = dem.rio.clip_box(bounds[0],bounds[1],bounds[2],bounds[3])
landfire2 = cbd.rio.clip_box(bounds[0],bounds[1],bounds[2],bounds[3]).rio.reproject_match(bool_img)
# landfire3 = evc.rio.clip_box(bounds[0],bounds[1],bounds[2],bounds[3])
# landfire4 = evh.rio.clip_box(bounds[0],bounds[1],bounds[2],bounds[3])
# landfire5 = f40.rio.clip_box(bounds[0],bounds[1],bounds[2],bounds[3])
# landfire6 = fvh.rio.clip_box(bounds[0],bounds[1],bounds[2],bounds[3])

data_dict = {'cbd': landfire2,
                 'fire_bool' : bool_img.bool, 
             'fire_frp' : frp_img.frp}
    
dem_goes = goes.rio.reproject_match(landfire2) 

dem_goes = dem_goes.assign(data_dict)

for var in list(dem_goes.data_vars):
    try:
        del dem_goes[var].attrs['grid_mapping']
    except KeyError:
        bads.append(var)
        
dem_goes.to_netcdf("temp/matched/dem_goes.nc")

landfire2.to_netcdf("temp/matched/landfire2.nc")


In [ ]:
for var in list(bool_img.data_vars):
    try:
        del bool_img[var].attrs['grid_mapping']
    except KeyError:
        bads.append(var)

bool_img.to_netcdf("temp/matched/boolimg.nc")

In [ ]:
import warnings
warnings.filterwarnings('ignore')


In [ ]:
CMI_C01

In [ ]:
def feature_engineer(img):
    feat1 = img['CMI_C06'] / img['CMI_C05']
    feat2 = img.CMI_C07 / img.CMI_C05
    feat3 = img.CMI_C07 / img.CMI_C06
    feat4 = img.CMI_C14 / img.CMI_C07
    
    data_dict = {'6_5': feat1, '7_5': feat2,
                 '7_6': feat3, '14_7': feat4}
    
    img_crs = img.rio.crs
    
    return img.assign(data_dict).rio.write_crs(img_crs)

In [ ]:
feature_engineer(goes)

In [ ]:
missing_goes

In [ ]:
i

In [ ]:
years

In [ ]:
viirs_files[4000:]

In [ ]:
bool_filt = [f.endswith('2017-12-30-19_Num1_bool.nc') for f in viirs_files]

bool_filt.index(True)
# viirs_files

In [ ]:
missing_goes = []
i = 0
years = ['2016']
for VIIRS_bool in viirs_files:
    bool_img = xr.open_dataset(VIIRS_bool)
    bool_img = bool_img.rio.write_crs('EPSG:4326')
    
    
    
    bounds = bool_img.rio.reproject(dem.rio.crs).rio.bounds()
    
    try: # Catch not CONUS data
        landfire1 = dem.rio.clip_box(bounds[0],bounds[1],bounds[2],bounds[3]).rio.reproject_match(bool_img)
        landfire2 = cbd.rio.clip_box(bounds[0],bounds[1],bounds[2],bounds[3]).rio.reproject_match(bool_img)
        landfire3 = evc.rio.clip_box(bounds[0],bounds[1],bounds[2],bounds[3]).rio.reproject_match(bool_img)
        landfire4 = evh.rio.clip_box(bounds[0],bounds[1],bounds[2],bounds[3]).rio.reproject_match(bool_img)
        landfire5 = f40.rio.clip_box(bounds[0],bounds[1],bounds[2],bounds[3]).rio.reproject_match(bool_img)
        landfire6 = fvh.rio.clip_box(bounds[0],bounds[1],bounds[2],bounds[3]).rio.reproject_match(bool_img)
        
    except RuntimeError:
        continue
 
    
    try:
        goes_path = get_goes_from_viirs(VIIRS_bool)
        goes = rioxarray.open_rasterio(goes_path)
        date = get_goes_time(goes_path)
        suffix = goes_path[-21:]
        year = str(date.year)
        if years[-1] != year:
            print('===========================')
            print(year + " STARTED")
        
        print("Opened File.")

        goes = feature_engineer(goes)
        dem_goes = goes.rio.reproject_match(landfire1)
    
        print("Reprojected_matched.")
        data_dict = {'dem': landfire1, 'cbd': landfire2,
                 'evc': landfire3, 'evh': landfire4,
                 'f40': landfire5, 'fvh': landfire6,
                 'fire_bool' : bool_img.bool}
    
        big_match = dem_goes.assign(data_dict)
    
    
        print("Assigned data.")
        # Run through xarray olympics to export
        bads = []
        vars_list = list(big_match.data_vars)  
        
        for var in vars_list:  
            try:
                del big_match[var].attrs['grid_mapping']
            except KeyError:
                bads.append(var)
        
        

        output_path = f'../Data/Training/Stacked/bool/take2/{year}/'
        
        if not os.path.exists(output_path):
            os.mkdir(output_path)
            
        big_match.to_netcdf(output_path+suffix)
        
        print("Exported data")
        print(f'{suffix} done.')
        
        
        print('\n')
        print("---------")
        i += 1
        year = str(int(year)+1)
        years.append(year)
        
    except OSError:
        missing_goes.append(VIIRS_bool)
        continue
    
    
    
    

In [ ]:
big_match['6/5']

# Code to get list of viirs files

In [ ]:
files = []
for r,d,f in os.walk('../Data/Training/VIIRS/Rasterized/Individual_Fires/variable_size/'):
    for file in f:
        if file.endswith('_bool.nc'):
            files.append(os.path.join(r,file))
#     print(folder)

I wanted to see where the largest VIIRS files were, to perhaps pad each viirs file to be the same exact size

In [ ]:
dims = []
for f in files:
    data = xr.open_dataset(f)
    x,y = data.dims['x'], data.dims['y']
    dims.append((x,y))
    data = None

xs = []
ys = []
for x,y in dims:
    xs.append(x)
    ys.append(y)

In [ ]:
dims[56]

In [ ]:
dims[335]

In [ ]:
max_x = xs.index(max(xs))
max_y = ys.index(max(ys))

print("dimensions of maxes: ")
print(dims[max_x])
print(dims[max_y])

print('indexes of maxes:')
print(dims.index(dims[max_x]))
print(dims.index(dims[max_y]))


In [ ]:
from rioxarray.merge import merge_arrays


raster1 = rioxarray.open_rasterio(files[418])
raster2 = rioxarray.open_rasterio(files[616])
raster_to_mosaic = [raster1, raster2]
merged = merge_arrays(raster_to_mosaic)



In theory, "merged" is a raster whose bounds encompass the bounds of every single VIIRS observation. Although this is difficult to verify using QGIS since it is so large and the fires are so small

# Need to down sampling all imagery to the smallest resolution of data products we have

https://pygis.io/docs/e_raster_resample.html

# Scratch

( from https://corteva.github.io/rioxarray/stable/examples/reproject_match.html)

In [ ]:
def print_raster(raster):
    print(
        f"shape: {raster.rio.shape}\n"
        f"resolution: {raster.rio.resolution()}\n"
        f"bounds: {raster.rio.bounds()}\n"
        f"CRS: {raster.rio.crs}\n"
    )

In [ ]:
fig, axes = plt.subplots(ncols=2, figsize=(12,4))
goes['CMI_C01'].plot(ax=axes[0])
viirs['band_data'].plot(ax=axes[1])
plt.draw()

In [ ]:
print("Original Raster:\n----------------\n")
print_raster(goes)
print("Raster to Match:\n----------------\n")
print_raster(viirs)

In [ ]:
goes_rio = rioxarray.open_rasterio(goes_path)

In [ ]:
goes_rio = rioxarray.open_rasterio(goes_path)
match = goes_rio.rio.reproject_match(viirs.fillna(0))

In [ ]:
match = match.assign_coords({
    "x": match.x,
    "y": match.y,
})


In [ ]:
vars_list = list(match.data_vars)  
bads = []
for var in vars_list:  
#     print(goes_viirs[var].attrs)
    try:
        del match[var].attrs['grid_mapping']
    except KeyError:
        bads.append(var)

In [ ]:
match.to_netcdf('temp/match.nc')

# Scratch 2

This is how I add bands to GOES imagery. Need to see if this works after using reproject match

In [ ]:
goes_viirs = goes2.assign({'fire_bool':viirs.rio.reproject('EPSG:4326').band_data})

vars_list = list(goes_viirs.data_vars)  
bads = []
for var in vars_list:  
#     print(goes_viirs[var].attrs)
    try:
        del goes_viirs[var].attrs['grid_mapping']
    except KeyError:
        bads.append(var)

In [ ]:
goes.assign({'fire_bool':viirs.band_data}).to_netcdf("temp/raw_goesviirs.nc")

In [ ]:
viirs.band_data['spatial_ref']